<a href="https://colab.research.google.com/github/Th3nn3ss/ai_chatbot/blob/main/training.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [25]:
from google.colab import files
upload = files.upload()


Saving intents.json to intents (2).json


In [37]:
import random
import json
import pickle
import numpy as np


import nltk
nltk.download("punkt")
nltk.download("wordnet")
from nltk.stem import WordNetLemmatizer

from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Activation, Dropout
from tensorflow.keras.optimizers import SGD


lemmatizer = WordNetLemmatizer()
intents = json.loads(open("intents (2).json").read())


words = []
classes = []
documents = []
ignore_letters = ["?", "!", ".", ",", ";", ";", "'"]

for intent in intents["intents"]:
  for pattern in intent["patterns"]:
    word_list = nltk.word_tokenize(pattern)
    words.extend(word_list)
    documents.append((word_list, intent['tag']))
    if intent["tag"] not in classes:
      classes.append(intent["tag"])

words = [lemmatizer.lemmatize(word) for word in words if word not in ignore_letters]
words = sorted(set(words))

classes = sorted(set(classes))

pickle.dump(words, open('words.pkl', 'wb'))
pickle.dump(classes, open('classes.pkl', 'wb'))

training = []
output_empt  = [0]*len(classes)

for document in documents:
  bag = []
  word_patterns = document[0]
  word_patterns = [lemmatizer.lemmatize(word.lower()) for word in word_patterns]
  for word in words:
    bag.append(1) if word in word_patterns else bag.append(0)

    output_row = list(output_empt)
    output_row[classes.index(document[1])] = 1
    training.append([bag, output_row])

random.shuffle(training)
training = np.array(training)

train_x = list(training[:, 0])
train_y = list(training[:, 1])

model = Sequential()
model.add(Dense(128, input_shape=(len(train_x[0]), ), activation="relu"))
model.add(Dropout(0.5))
model.add(Dense(64, activation="relu"))
model.add(Dropout(0.5))
model.add(Dense(len(train_y[0]), activation="softmax"))

sgd = SGD(lr=0.01, decay=1e-6, momentum=0.9, nesterov=True)
model.compile(loss='categorical_crossentropy', optimizer=sgd, metrics=['accuracy'])

hist = model.fit(np.array(train_x), np.array(train_y), epochs=200, batch_size=5, verbose=1 )
model.save("chatbotmodel.h5", hist)
print("Done")

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
Epoch 1/200


/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:57: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray


492/492 [==============================] - 1s 1ms/step - loss: 1.1719 - accuracy: 0.5550
Epoch 2/200
492/492 [==============================] - 1s 1ms/step - loss: 0.1023 - accuracy: 0.9752
Epoch 3/200
492/492 [==============================] - 1s 1ms/step - loss: 0.0337 - accuracy: 0.9940
Epoch 4/200
492/492 [==============================] - 1s 1ms/step - loss: 0.0262 - accuracy: 0.9948
Epoch 5/200
492/492 [==============================] - 1s 1ms/step - loss: 0.0156 - accuracy: 0.9964
Epoch 6/200
492/492 [==============================] - 1s 1ms/step - loss: 0.0121 - accuracy: 0.9976
Epoch 7/200
492/492 [==============================] - 1s 1ms/step - loss: 0.0144 - accuracy: 0.9976
Epoch 8/200
492/492 [==============================] - 1s 1ms/step - loss: 0.0050 - accuracy: 0.9997
Epoch 9/200
492/492 [==============================] - 1s 1ms/step - loss: 0.0080 - accuracy: 0.9993
Epoch 10/200
492/492 [==============================] - 1s 1ms/step - loss: 0.0086 - accuracy: 0.9984
E

In [43]:
from tensorflow.keras.models import load_model
words = pickle.load(open("words.pkl", "rb"))
classes = pickle.load(open("classes.pkl", "rb"))
model = load_model("chatbotmodel.h5")

def clean_up_sentence(sentence):
  sentence_words = nltk.word_tokenize(sentence)
  sentence_words = [lemmatizer.lemmatize(word) for word in sentence_words]
  return sentence_words

def bag_of_words(sentence):
  sentence_words = clean_up_sentence(sentence)
  bag = [0]*len(words)
  for w in sentence_words:
    for i, word in enumerate(words):
      if word == w:
        bag[i] = 1
    return np.array(bag)

def predict_class(sentence):
  bow = bag_of_words(sentence)
  res = model.predict(np.array([bow]))[0]
  ERRO_THRESHOLD = 0.25
  results = [(i, r) for i, r in enumerate(res) if r > ERRO_THRESHOLD]

  results.sort(key=lambda x: x[1], reverse=True)
  return_list = []
  for r in results:
    return_list.append({'intent': classes[r[0]], 'probability': str(r[1])})
  return return_list

def get_response(intents_list, intents_json):
  tag = intents_list[0]['intent']
  list_of_intents = intents_json['intents']
  for i in list_of_intents:
    if i['tag'] == tag:
      result = random.choice(i['responses'])
      break
  return result

while True:
  message = input("")
  ints = predict_class(message)
  res = get_response(ints, intents)
  print(res)

KeyboardInterrupt: ignored